# How we queried the database

This is an example of how we queried the DES database. It's useful to understand how the query works, but you can download the results directly because I saved the .csv files.

The most useful one is des_dr1_g_i_small_box_hpix_4.csv because I appended hpix IDs with nside = 4 to each star.

In [1]:
# std lib
from collections import OrderedDict
from getpass import getpass
import warnings
from matplotlib.colors import LogNorm
warnings.filterwarnings('ignore') # to suppress some astropy deprecation warnings

# 3rd party
import numpy as np
import pylab as plt
from astropy import utils, io, convolution, wcs, stats
from astropy.visualization import make_lupton_rgb
from astropy.coordinates import name_resolve
from pyvo.dal import sia
import pandas as pd
from scipy.ndimage.filters import gaussian_filter
from photutils import find_peaks
from scipy.spatial import cKDTree
import healpy
from sklearn.cluster import DBSCAN

%matplotlib inline

# Data Lab
from dl import authClient as ac, queryClient as qc, storeClient as sc
from dl.helpers.utils import convert
from dl.helpers.crossmatch import xmatch

# set up Simple Image Access (SIA) service
DEF_ACCESS_URL = "http://datalab.noao.edu/sia/des_dr1"
svc = sia.SIAService(DEF_ACCESS_URL)

In [2]:
# To save to virtual space, need to log in your account (not anonymous)
token = ac.login(input('Enter user name (+ENTER): '),getpass('Enter password (+ENTER): '))  

Enter user name (+ENTER): zilishen
Enter password (+ENTER): ········


In [4]:
sc.put (fr='./Querying_database_g_i_box.ipynb', to = 'vos://public/des/')

(1 / 1) ./Querying_database_g_i_box.ipynb -> vos://public/des/Querying_database_g_i_box.ipynb


'OK'

In [5]:
sc.put (fr='./des_dr1_g_i_small_box_hpix_4.csv', to = 'vos://public/des/')

(1 / 1) ./des_dr1_g_i_small_box_hpix_4.csv -> vos://public/des/des_dr1_g_i_small_box_hpix_4.csv


'OK'

# Query the database

In [6]:
# The DES DR1 Schema
print(qc.schema('des_dr1',format='',profile='default'))
#print(main.columns)


Schema: des_dr1

      Table Name   Description
      ----------   -----------
     des_allwise   ALLWISE 1-arcsec xmatch v. DES
       des_gaia2   Gaia DR2 1.5-arcsec xmatch v. DES
       des_galex   GALEX (AIS) 1-arcsec xmatch v. DES
        des_hsc2   HSC2 1-arcsec xmatch v. DES
      des_nscdr1   NSC DR1 1.5-arcsec xmatch v. DES
      des_simbad   SIMBAD 1-arcsec xmatch v. DES
            flux   Object flux table
        galaxies   Pure galaxy sample
       img2coadd   Images contributing to coadded tiles
             mag   Object magnitude table
            main   Main object table
       neighbors   Neighbors w/in 30-arcsec of each object
           stars   Pure star sample
       tile_info   Survey tile information table
  x_gaia_dr2_2p5   Gaia DR1 2.5 arcsec xmatch v. DES



In [8]:
# columns to query for
columns = '''ra,dec,mag_auto_g,mag_auto_i,magerr_auto_g,magerr_auto_i,tilename,hpix_1024,hpix_32,hpix_4096,hpix_64'''

In [9]:
# a function to retrieve data
def getData(columns='*'):

    query_template =\
    """SELECT {0} FROM des_dr1.main
       WHERE spread_model_r + 3.0*spreaderr_model_r < 0.003 AND spread_model_r > -0.1 AND 
       flags_r < 4.0 AND flags_g < 4.0 AND
       imaflags_iso_r = 0 AND imaflags_iso_g = 0 AND
       (mag_auto_g - mag_auto_i) < 0.0 AND (mag_auto_g - mag_auto_i) > -0.9 AND
       mag_auto_g < 25.5 AND mag_auto_g > 17
       """
    # r-band has the better morphology classification than other bands
    # i-band is better than r-band, but shallower
    # In querying, using "between" is inefficient

    query = query_template.format(columns)

    
    return query

# big box from isochrone: 17 < g < 26, -0.9 < g-i < 0.2
# smaller box: 17 < g < 25.5, -0.9 < g-i < 0.0

In [10]:
query = getData(columns)
### Simple query
#df = qc.query(query, fmt='pandas')

In [11]:
### Using async because this query may take a while
jobid = qc.query(query,async=True)
print (jobid)

h2e9z05rsehh5b91


In [17]:
status = qc.status(jobid)
print (status)

COMPLETED


In [18]:
df = convert(qc.results(jobid),'pandas')
df.head()

,ra,dec,mag_auto_g,mag_auto_i,magerr_auto_g,magerr_auto_i,tilename,hpix_1024,hpix_32,hpix_4096,hpix_64
0,326.893500,-39.919220,23.179918,23.469385,0.231884,0.644059,DES2148-3957,11977166,11696,191634667,46785
1,326.941361,-39.789704,17.020658,17.383627,0.001475,0.003444,DES2148-3957,11977200,11696,191635209,46785
2,327.339690,-40.030502,22.474710,22.546764,0.080204,0.174881,DES2148-3957,11977078,11696,191633257,46785
3,327.196766,-39.860714,17.485518,17.717770,0.001582,0.003320,DES2148-3957,11977172,11696,191634766,46785
4,327.519006,-40.082185,21.122114,21.316036,0.018762,0.046286,DES2148-3957,11977055,11696,191632885,46785


In [19]:
df.shape

(305817, 11)

In [20]:
df.to_csv('des_dr1_g_i_small_box_hpix.csv',index=False) # Save the table to your virtual space